In [1]:
import os
import numpy as np
import librosa
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# ------------------ Feature Extraction ------------------
def pad_or_truncate(features, target_length):
    """Pad or truncate features to a fixed length."""
    if features.shape[0] < target_length:
        padding = target_length - features.shape[0]
        padded_features = np.pad(features, ((0, padding), (0, 0)), mode='constant')
        return padded_features
    else:
        return features[:target_length, :]

def extract_lp_residual(audio, order=12):
    """Extract Linear Prediction (LP) residual"""
    lpc_coeffs = librosa.lpc(audio, order=order)
    lp_residual = np.convolve(audio, -lpc_coeffs, mode='full')[:len(audio)]
    lp_residual = np.nan_to_num(lp_residual, nan=0.0, posinf=0.0, neginf=0.0)
    return lp_residual

def extract_mgdcc(audio, sr, alpha=0.4, gamma=0.9, num_coeffs=13):
    """Extract Modified Group Delay Cepstral Coefficients (MGDCC)"""
    stft = librosa.stft(audio)
    magnitude = np.abs(stft)
    phase = np.angle(stft)
    group_delay = -np.diff(phase, axis=0)
    modified_gd = group_delay * (magnitude[:-1, :] ** gamma)
    mgd_cepstral = librosa.feature.mfcc(S=modified_gd, n_mfcc=num_coeffs)
    return mgd_cepstral.T

In [2]:
# ------------------ Dataset Loader ------------------
class DysarthriaDataset(Dataset):
    def __init__(self, data_path, severity_mapping, sr=16000, target_length=100):
        self.data_path = data_path
        self.severity_mapping = severity_mapping
        self.sr = sr
        self.target_length = target_length
        self.data = []
        self.labels = []

        severity_to_label = {severity: i for i, severity in enumerate(severity_mapping.keys())}

        for severity, speakers in severity_mapping.items():
            for speaker in speakers:
                speaker_path = os.path.join(data_path, speaker)
                for file in os.listdir(speaker_path):
                    if file.endswith(".wav"):
                        self.data.append(os.path.join(speaker_path, file))
                        self.labels.append(severity_to_label[severity])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        file_path = self.data[idx]
        label = self.labels[idx]
        audio, _ = librosa.load(file_path, sr=self.sr)
        lp_residual = extract_lp_residual(audio)
        features = extract_mgdcc(lp_residual, self.sr)
        features = pad_or_truncate(features, self.target_length)
        features = torch.tensor(features, dtype=torch.float32).unsqueeze(0)  # Add channel dimension
        label = torch.tensor(label, dtype=torch.long)
        return features, label

In [3]:
class DysarthriaSeverityCNN(nn.Module):
    def __init__(self, num_classes=4):
        super(DysarthriaSeverityCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc1_input_size = None  # Placeholder for dynamic computation
        self.fc1 = None
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        
        # Flatten the output of the convolutional layers
        x = x.view(x.size(0), -1)

        # Dynamically initialize fc1 if needed
        if self.fc1 is None:
            self.fc1_input_size = x.size(1)
            self.fc1 = nn.Linear(self.fc1_input_size, 128).to(x.device)

        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)  # Raw logits
        return x

In [4]:
def train_model(model, dataloader, criterion, optimizer, device, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0
        correct = 0
        total = 0

        for inputs, labels in tqdm(dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
            # Move data to device
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        if (epoch+1)%10 == 0:
            torch.save(model, f"model_{epoch+1}.pth")
            print(f"Model saved at {epoch+1+25}")
        accuracy = 100 * correct / total
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss / len(dataloader):.4f}, Accuracy: {accuracy:.2f}%")

In [5]:
from torch.utils.data import random_split

severity_mapping = {
    "HIGH": ["M01", "M04", "M12", "F03"],
    "MEDIUM": ["F02", "M07", "M16"],
    "LOW": ["F04", "M05", "M11"],
    "VERY LOW": ["F05", "M08", "M09", "M10", "M14"]
}

dataset_path = "/kaggle/input/dysarthria-data/noisereduced-uaspeech"
dataset = DysarthriaDataset(dataset_path, severity_mapping)
# Splitting into train and eval sets
train_size = int(0.8 * len(dataset))  # 80% training
eval_size = len(dataset) - train_size  # 20% evaluation
train_dataset, eval_dataset = random_split(dataset, [train_size, eval_size])

# Creating DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=6, pin_memory=True)
eval_dataloader = DataLoader(eval_dataset, batch_size=64, shuffle=False, num_workers=6, pin_memory=True)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [11]:
# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Make sure the DysarthriaSeverityCNN class is defined in the script
# Load the entire DataParallel object
data_parallel_model = torch.load("/kaggle/input/models-dysar/model_70.pth")

# Extract the actual model from the DataParallel wrapper
model = data_parallel_model.module

# Move the model to the correct device
model = model.to(device)

# Wrap it again with DataParallel if using multiple GPUs
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)

# Check model with dummy input to initialize layers dynamically
dummy_input = torch.zeros(1, 1, 100, 13).to(device)  # Shape matches input feature maps
model(dummy_input)

# Wrap model in DataParallel if multiple GPUs are available
# if torch.cuda.device_count() > 1:
#     print(f"Using {torch.cuda.device_count()} GPUs!")
#     model = nn.DataParallel(model)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

<ipython-input-11-fad31d6b6849>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data_parallel_model = torch.load("/kaggle/input/models-dysar/model_70.pth")


In [45]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.current_device())  # Check current GPU device
print(torch.cuda.get_device_name(0))  # Get GPU name
print(torch.cuda.get_device_name(1))  # Get GPU name

True
0
Tesla T4
Tesla T4


In [7]:
model = model.to(device)  # where device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
device

device(type='cuda')

In [67]:
# Train the model
print("Starting training...")
train_model(
    model=model,
    dataloader=train_dataloader,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    num_epochs=75
)

Starting training...


Epoch 1/75: 100%|██████████| 921/921 [04:12<00:00,  3.65it/s]


Epoch 1/75, Loss: 0.5173, Accuracy: 85.16%


Epoch 2/75: 100%|██████████| 921/921 [04:03<00:00,  3.78it/s]


Epoch 2/75, Loss: 0.3789, Accuracy: 86.52%


...

Epoch 73/75, Loss: 0.1397, Accuracy: 94.90%


Epoch 74/75: 100%|██████████| 921/921 [04:23<00:00,  3.49it/s]


Epoch 74/75, Loss: 0.1267, Accuracy: 95.34%


Epoch 75/75: 100%|██████████| 921/921 [04:24<00:00,  3.49it/s]

Epoch 75/75, Loss: 0.1234, Accuracy: 95.38%


In [29]:
# Save only the state dictionary
torch.save(model.module.state_dict(), "model_50.pth")


In [34]:
import shutil
from IPython.display import HTML

In [36]:
def create_download_link(filename):
    shutil.move(filename, f"/kaggle/working/{filename}")
    return HTML(f'<a href="{filename}" download>{filename}</a>')

# Generate the download link for the saved model
create_download_link("model.pth")

In [12]:
# Evaluation loop
def evaluate_model(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Evaluating"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    avg_loss = total_loss / len(dataloader)
    print(f"Validation Loss: {avg_loss:.4f}, Validation Accuracy: {accuracy:.2f}%")
    return avg_loss, accuracy

# Evaluate the model on validation set
print("Evaluating model...")
evaluate_model(model, eval_dataloader, criterion, device)

Evaluating model...


Evaluating: 100%|██████████| 231/231 [01:05<00:00,  3.54it/s]

Validation Loss: 0.1598, Validation Accuracy: 97.31%


(0.1598398262358702, 97.3057346454021)